# RSNA 2024 Lumbar Spine Degenerative Classification
## Version 5 — Redesigned Pipeline

### Key Changes from v4 → v5:
1. **WeightedRandomSampler** replaces oversampling (balanced batches, no duplicates)
2. **DICOM Windowing** preserves clinical contrast (uses Window/Level metadata)
3. **Attention Pooling** replaces mean pooling (learns which frames matter)
4. **FiLM Conditioning** for level embedding (modulates features, not concatenates)
5. **Gradient Clipping** + reduced label smoothing for stable training
6. **Cosine Warm Restarts** for LR schedule (escape local minima)
7. **SWA** in final epochs for smoother generalization
8. **TTA** at inference for +1-2% balanced accuracy


In [ ]:
import os
import copy
import cv2
import glob
import pydicom
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
from tqdm import tqdm
from sklearn.model_selection import StratifiedGroupKFold
from sklearn.metrics import confusion_matrix, classification_report, balanced_accuracy_score
from collections import Counter


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from torch.amp import autocast, GradScaler
from torch.optim.swa_utils import AveragedModel, SWALR
import albumentations as A
from albumentations.pytorch import ToTensorV2


In [ ]:
# --- CONFIGURATION v5 ---
CONFIG = {
    'seed': 42,
    'img_size': 256,
    'seq_length': 7,
    'batch_size': 8,
    'epochs': 30,
    'learning_rate': 3e-4,
    'backbone_lr': 3e-5,
    'weight_decay': 0.05,
    'patience': 12,
    'num_folds': 5,
    'train_folds': [0],
    
    # Loss — simplified: no class weights, sampler handles balance
    'focal_gamma': 2.0,
    'label_smoothing': 0.05,       # Reduced from 0.1
    
    # Training stability
    'clip_grad_norm': 1.0,         # NEW: gradient clipping
    'use_swa': True,               # NEW: stochastic weight averaging
    'swa_start_epoch': 20,         # Start SWA after epoch 20
    'swa_lr': 1e-5,                # SWA learning rate
    
    # Architecture
    'hidden_dim': 256,
    'dropout': 0.4,
    'num_attention_heads': 4,
    'stochastic_depth_rate': 0.1,  # NEW: drop path for backbone
    
    # Scheduler
    'warmup_epochs': 2,
    'T_0': 8,                      # NEW: cosine warm restart period
    'T_mult': 2,                   # NEW: period multiplier
    
    # Data
    'min_minority_recall': 0.15,
    'use_mixup': True,             # NEW
    'mixup_alpha': 0.3,            # NEW
    
    'device': torch.device('cuda' if torch.cuda.is_available() else 'cpu'),
    'target_condition': 'spinal_canal_stenosis',
    'target_series': 'Sagittal T2/STIR'
}


In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

seed_everything(CONFIG['seed'])
print(f"✅ Device: {CONFIG['device']}")
print(f"   Version: 5 (Redesigned Pipeline)")
print(f"   Focal Gamma: {CONFIG['focal_gamma']}, Label Smoothing: {CONFIG['label_smoothing']}")
print(f"   Gradient Clipping: {CONFIG['clip_grad_norm']}")
print(f"   SWA: {CONFIG['use_swa']} (from epoch {CONFIG['swa_start_epoch']})")
print(f"   Mixup: {CONFIG['use_mixup']} (alpha={CONFIG['mixup_alpha']})")


## 1. Data Loading

In [ ]:
# --- PATHS ---
DATA_ROOT = "/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/"
TRAIN_IMAGES = os.path.join(DATA_ROOT, "train_images")


In [ ]:
df_train = pd.read_csv(f"{DATA_ROOT}/train.csv")
df_coords = pd.read_csv(f"{DATA_ROOT}/train_label_coordinates.csv")
df_desc = pd.read_csv(f"{DATA_ROOT}/train_series_descriptions.csv")


In [ ]:
# Clean & Merge
df_train.columns = [col.lower().replace('/', '_') for col in df_train.columns]
condition_cols = [c for c in df_train.columns if c != 'study_id']
df_labels = pd.melt(df_train, id_vars=['study_id'], value_vars=condition_cols,
                    var_name='condition_level', value_name='severity')
df_labels = df_labels.dropna(subset=['severity'])
df_labels['severity'] = df_labels['severity'].astype(str).str.lower().str.replace('/', '_')


In [ ]:
def extract_meta(val):
    parts = val.split('_')
    level = parts[-2] + '_' + parts[-1]
    condition = '_'.join(parts[:-2])
    return condition, level


In [ ]:
df_labels[['base_condition', 'level_str']] = df_labels['condition_level'].apply(lambda x: pd.Series(extract_meta(x)))
severity_map = {'normal_mild': 0, 'moderate': 1, 'severe': 2}
df_labels['label'] = df_labels['severity'].map(severity_map)
df_labels = df_labels.dropna(subset=['label'])
df_labels['label'] = df_labels['label'].astype(int)


In [ ]:
df_coords = df_coords.merge(df_desc, on=['study_id', 'series_id'], how='left')
df_coords['condition'] = df_coords['condition'].str.lower().str.replace(' ', '_')
df_coords['level'] = df_coords['level'].str.lower().str.replace('/', '_')
df_coords['condition_level'] = df_coords['condition'] + '_' + df_coords['level']


In [ ]:
df_model = df_labels[df_labels['base_condition'] == CONFIG['target_condition']].copy()
df_coords_filt = df_coords[(df_coords['condition'] == CONFIG['target_condition']) & 
                           (df_coords['series_description'] == CONFIG['target_series'])]


In [ ]:
df_final = df_model.merge(df_coords_filt[['study_id', 'condition_level', 'series_id', 'instance_number', 'x', 'y']],
                          on=['study_id', 'condition_level'], how='inner')


In [ ]:
# Filter valid files
valid_rows = []
for index, row in tqdm(df_final.iterrows(), total=len(df_final), desc="Checking Files"):
    path = f"{TRAIN_IMAGES}/{row['study_id']}/{row['series_id']}/{int(row['instance_number'])}.dcm"
    if os.path.exists(path):
        valid_rows.append(row)


In [ ]:
df_final = pd.DataFrame(valid_rows).reset_index(drop=True)
level_map = {'l1_l2': 0, 'l2_l3': 1, 'l3_l4': 2, 'l4_l5': 3, 'l5_s1': 4}
df_final['level_idx'] = df_final['level_str'].map(level_map)

print(f"\n✅ Data Ready: {len(df_final)} samples")
print(f"   Class Distribution: {df_final['label'].value_counts().sort_index().to_dict()}")
class_counts = df_final['label'].value_counts().sort_index()
for i, count in enumerate(class_counts):
    pct = count / len(df_final) * 100
    print(f"   Class {i}: {count} samples ({pct:.1f}%)")


## 2. Weighted Random Sampler (Replaces Oversampling)

**Why this is better than oversampling:**
- Every batch has approximately equal class representation
- No duplicate rows in the DataFrame (less memorization)
- Each sample gets a **different** augmentation every time it's drawn
- No global seed corruption from `np.random.seed` in `__getitem__`


In [ ]:
def create_weighted_sampler(df):
    """Create a WeightedRandomSampler for balanced class sampling."""
    class_counts = np.bincount(df['label'].values, minlength=3).astype(float)
    # Inverse frequency weighting
    class_weights = 1.0 / class_counts
    sample_weights = class_weights[df['label'].values]
    
    sampler = WeightedRandomSampler(
        weights=sample_weights,
        num_samples=len(df),
        replacement=True
    )
    
    print(f"\n📊 WeightedRandomSampler created:")
    print(f"   Class counts: {class_counts.astype(int).tolist()}")
    print(f"   Class weights: [{', '.join(f'{w:.4f}' for w in class_weights)}]")
    print(f"   Effective sampling: each class drawn ~equally per epoch")
    
    return sampler


## 3. Dataset with Fixed DICOM Windowing

In [ ]:
class RSNASequenceDatasetV5(Dataset):
    """
    v5 improvements:
    - DICOM windowing preserves clinical contrast
    - Crop jittering for training robustness
    - No global seed corruption
    - Supports Mixup via returning float labels
    """
    def __init__(self, df, seq_length=7, img_size=256, transform=None, is_training=False):
        self.df = df.reset_index(drop=True)
        self.seq_length = seq_length
        self.img_size = img_size
        self.transform = transform
        self.is_training = is_training
        self.clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
        
    def __len__(self):
        return len(self.df)
    
    def load_dicom(self, path):
        """Load DICOM with proper windowing (Tier 1 fix)."""
        try:
            dcm = pydicom.dcmread(path)
            img = dcm.pixel_array.astype(np.float32)
            
            # Use DICOM Window/Level when available
            if hasattr(dcm, 'WindowCenter') and hasattr(dcm, 'WindowWidth'):
                wc = dcm.WindowCenter
                ww = dcm.WindowWidth
                # Handle MultiValue
                if isinstance(wc, pydicom.multival.MultiValue):
                    wc = float(wc[0])
                else:
                    wc = float(wc)
                if isinstance(ww, pydicom.multival.MultiValue):
                    ww = float(ww[0])
                else:
                    ww = float(ww)
                # Apply windowing
                img = np.clip((img - (wc - ww/2)) / max(ww, 1) * 255, 0, 255)
            else:
                # Fallback to min-max
                if img.max() > img.min():
                    img = (img - img.min()) / (img.max() - img.min()) * 255.0
                else:
                    img = np.zeros_like(img)
            
            img = img.astype(np.uint8)
            img = self.clahe.apply(img)
            return img
        except:
            return np.zeros((self.img_size, self.img_size), dtype=np.uint8)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        center_inst = int(row['instance_number'])
        study_path = f"{TRAIN_IMAGES}/{row['study_id']}/{row['series_id']}"
        cx, cy = int(row['x']), int(row['y'])
        
        # Crop jittering during training (Tier 1 fix)
        if self.is_training:
            jitter = self.img_size // 20  # ~5% jitter
            cx += random.randint(-jitter, jitter)
            cy += random.randint(-jitter, jitter)
        
        start = center_inst - (self.seq_length // 2)
        indices = [start + i for i in range(self.seq_length)]
        
        images_list = []
        for inst in indices:
            path = os.path.join(study_path, f"{inst}.dcm")
            if os.path.exists(path):
                img = self.load_dicom(path)
            else:
                img = np.zeros((self.img_size, self.img_size), dtype=np.uint8)
            
            h, w = img.shape
            crop_size = self.img_size // 2 
            x1 = max(0, cx - crop_size)
            y1 = max(0, cy - crop_size)
            x2 = min(w, cx + crop_size)
            y2 = min(h, cy + crop_size)
            crop = img[y1:y2, x1:x2]
            
            if crop.size == 0:
                crop = np.zeros((self.img_size, self.img_size), dtype=np.uint8)
            else:
                crop = cv2.resize(crop, (self.img_size, self.img_size))
            
            crop = cv2.cvtColor(crop, cv2.COLOR_GRAY2RGB)
            
            if self.transform:
                res = self.transform(image=crop)
            else:
                res = {'image': torch.tensor(crop).permute(2, 0, 1).float() / 255.0}
            
            images_list.append(res['image'])
            
        sequence = torch.stack(images_list, dim=0)
        label = torch.tensor(row['label'], dtype=torch.long)
        level_idx = torch.tensor(row['level_idx'], dtype=torch.long)
        
        return sequence, label, level_idx


## 4. Augmentation Pipelines

In [ ]:
# Single strong pipeline — sampler handles class balance, 
# so we don't need separate weak/strong augmentations
train_aug = A.Compose([
    A.ShiftScaleRotate(shift_limit=0.08, scale_limit=0.12, rotate_limit=8,
                       border_mode=cv2.BORDER_CONSTANT, value=0, p=0.6),
    A.OneOf([
        A.RandomBrightnessContrast(brightness_limit=0.25, contrast_limit=0.25, p=1.0),
        A.RandomGamma(gamma_limit=(80, 120), p=1.0),
        A.CLAHE(clip_limit=4.0, p=1.0),
    ], p=0.7),
    A.OneOf([
        A.GaussNoise(var_limit=(5.0, 30.0), p=1.0),
        A.MultiplicativeNoise(multiplier=(0.9, 1.1), p=1.0),
    ], p=0.3),
    A.ElasticTransform(alpha=1, sigma=50, alpha_affine=20, p=0.2),
    A.GridDistortion(num_steps=5, distort_limit=0.08, p=0.2),
    A.HorizontalFlip(p=0.5),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

val_aug = A.Compose([
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

# TTA augmentations
tta_augs = [
    val_aug,  # Original
    A.Compose([  # Horizontal flip
        A.HorizontalFlip(p=1.0),
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        ToTensorV2()
    ]),
]

print("✅ Augmentation pipelines ready")
print("   - Train: unified strong pipeline (sampler handles balance)")
print("   - Val: normalize only")
print(f"   - TTA: {len(tta_augs)} augmentation variants")


## 5. Model Architecture v5

**Key architectural changes:**
- **AttentionPool**: Learns which frames contain diagnostic information (replaces mean pooling)
- **FiLM Conditioning**: Level embedding modulates features instead of being concatenated
- **Simplified sequence model**: BiGRU (lighter than BiLSTM) without redundant self-attention
- **Stochastic depth**: Regularization on backbone


In [ ]:
class AttentionPool(nn.Module):
    """Learn to weight sequence frames by importance."""
    def __init__(self, dim):
        super().__init__()
        self.attn = nn.Sequential(
            nn.Linear(dim, dim // 4),
            nn.Tanh(),
            nn.Linear(dim // 4, 1)
        )
    
    def forward(self, x):
        # x: (B, seq_len, dim)
        weights = F.softmax(self.attn(x), dim=1)  # (B, seq_len, 1)
        pooled = (x * weights).sum(dim=1)          # (B, dim)
        return pooled, weights.squeeze(-1)          # Return weights for visualization


class FiLMLayer(nn.Module):
    """Feature-wise Linear Modulation for level conditioning."""
    def __init__(self, num_levels, feature_dim):
        super().__init__()
        self.gamma = nn.Embedding(num_levels, feature_dim)
        self.beta = nn.Embedding(num_levels, feature_dim)
        # Initialize gamma to 1, beta to 0 (identity transform)
        nn.init.ones_(self.gamma.weight)
        nn.init.zeros_(self.beta.weight)
    
    def forward(self, x, level_idx):
        # x: (B, feature_dim)
        g = self.gamma(level_idx)  # (B, feature_dim)
        b = self.beta(level_idx)   # (B, feature_dim)
        return g * x + b


class SpineModelV5(nn.Module):
    """
    Redesigned spine stenosis classifier:
    - EfficientNetV2-S backbone with stochastic depth
    - BiGRU sequence encoder (lighter than BiLSTM)
    - Attention pooling (learns which frames matter)
    - FiLM conditioning (level modulates features)
    """
    def __init__(self, num_classes=3, hidden_dim=256, gru_layers=2,
                 dropout=0.4, num_levels=5, stochastic_depth=0.1):
        super().__init__()
        
        # Backbone with stochastic depth
        effnet = models.efficientnet_v2_s(weights='IMAGENET1K_V1')
        
        # Apply stochastic depth to backbone blocks
        if stochastic_depth > 0:
            blocks = list(effnet.features.children())
            num_blocks = len(blocks)
            for i, block in enumerate(blocks):
                if hasattr(block, 'stochastic_depth'):
                    block.stochastic_depth.p = stochastic_depth * (i / num_blocks)
        
        self.backbone = nn.Sequential(*list(effnet.children())[:-1])
        self.feature_dim = 1280
        
        # Feature projection
        self.feature_proj = nn.Sequential(
            nn.Dropout(dropout),
            nn.Linear(self.feature_dim, hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.GELU()
        )
        
        # BiGRU sequence encoder (lighter than BiLSTM, similar performance)
        self.gru = nn.GRU(
            input_size=hidden_dim,
            hidden_size=hidden_dim // 2,
            num_layers=gru_layers,
            batch_first=True,
            bidirectional=True,
            dropout=dropout if gru_layers > 1 else 0
        )
        
        # Attention pooling (replaces mean pooling)
        self.attn_pool = AttentionPool(hidden_dim)
        
        # FiLM conditioning for level (replaces concatenation)
        self.film = FiLMLayer(num_levels, hidden_dim)
        
        # Classifier
        self.classifier = nn.Sequential(
            nn.LayerNorm(hidden_dim),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, 128),
            nn.GELU(),
            nn.Dropout(dropout * 0.5),  # Less dropout in final layer
            nn.Linear(128, num_classes)
        )
        
    def forward(self, x, level_idx=None):
        b, s, c, h, w = x.size()
        x = x.view(b * s, c, h, w)
        
        # Extract features
        features = self.backbone(x)
        features = features.view(b, s, -1)
        features = self.feature_proj(features)
        
        # Sequence encoding
        gru_out, _ = self.gru(features)
        
        # Attention pooling — learn which frames matter
        context, attn_weights = self.attn_pool(gru_out)
        
        # FiLM conditioning — level modulates features
        if level_idx is not None:
            context = self.film(context, level_idx)
        
        # Classification
        logits = self.classifier(context)
        
        return logits, attn_weights

print("✅ SpineModelV5 architecture defined")
print("   - Backbone: EfficientNet-V2-S with stochastic depth")
print("   - Sequence: BiGRU (lighter than BiLSTM)")
print("   - Pooling: Attention-weighted (not mean)")
print("   - Level: FiLM conditioning (modulates, not concatenates)")


## 6. Loss Functions

In [ ]:
class FocalLoss(nn.Module):
    """
    Focal Loss WITHOUT class weights.
    WeightedRandomSampler handles class balance at the data level.
    """
    def __init__(self, gamma=2.0, label_smoothing=0.05, reduction='mean'):
        super().__init__()
        self.gamma = gamma
        self.label_smoothing = label_smoothing
        self.reduction = reduction
        
    def forward(self, inputs, targets):
        ce_loss = F.cross_entropy(
            inputs, targets,
            reduction='none',
            label_smoothing=self.label_smoothing
        )
        pt = torch.exp(-ce_loss)
        focal_loss = ((1 - pt) ** self.gamma) * ce_loss
        
        if self.reduction == 'mean':
            return focal_loss.mean()
        return focal_loss


def mixup_data(x, y, alpha=0.3):
    """Mixup augmentation — blends samples for better generalization."""
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1.0
    
    batch_size = x.size(0)
    index = torch.randperm(batch_size, device=x.device)
    
    mixed_x = lam * x + (1 - lam) * x[index]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam


def mixup_criterion(criterion, pred, y_a, y_b, lam):
    """Compute loss for mixup samples."""
    return lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)

print("✅ FocalLoss (no class weights) + Mixup ready")


In [ ]:
class EarlyStopping:
    def __init__(self, patience=5, min_delta=0.001, mode='max'):
        self.patience = patience
        self.min_delta = min_delta
        self.mode = mode
        self.counter = 0
        self.best_score = None
        
    def __call__(self, val_score):
        if self.best_score is None:
            self.best_score = val_score
            return False
        
        improved = (val_score > self.best_score + self.min_delta) if self.mode == 'max'                    else (val_score < self.best_score - self.min_delta)
        
        if improved:
            self.best_score = val_score
            self.counter = 0
            return False
        
        self.counter += 1
        return self.counter >= self.patience


In [ ]:
def compute_per_class_metrics(preds, labels, num_classes=3):
    """Compute per-class recall."""
    metrics = {}
    for c in range(num_classes):
        mask = (labels == c)
        if mask.sum() > 0:
            correct = ((preds == c) & mask).sum()
            metrics[f'class_{c}_recall'] = correct / mask.sum()
        else:
            metrics[f'class_{c}_recall'] = 0.0
    return metrics


## 7. Training Function v5

**Changes from v4:**
- No class weights in loss (sampler handles balance)
- Gradient clipping for stability
- Cosine annealing with warm restarts
- SWA in final epochs
- Mixup augmentation
- Simplified model output (logits, attn_weights) — no aux dict


In [ ]:
def train_one_fold_v5(model, train_loader, val_loader, fold, config):
    """
    v5 Training function with all Tier 1+2 improvements.
    """
    criterion = FocalLoss(
        gamma=config['focal_gamma'],
        label_smoothing=config['label_smoothing']
    )
    
    optimizer = optim.AdamW([
        {'params': model.backbone.parameters(), 'lr': config['backbone_lr']},
        {'params': model.feature_proj.parameters(), 'lr': config['learning_rate']},
        {'params': model.gru.parameters(), 'lr': config['learning_rate']},
        {'params': model.attn_pool.parameters(), 'lr': config['learning_rate']},
        {'params': model.film.parameters(), 'lr': config['learning_rate']},
        {'params': model.classifier.parameters(), 'lr': config['learning_rate']},
    ], weight_decay=config['weight_decay'])
    
    # Cosine annealing with warm restarts
    warmup_steps = config['warmup_epochs'] * len(train_loader)
    total_steps = config['epochs'] * len(train_loader)
    
    def lr_lambda(step):
        if step < warmup_steps:
            return step / max(warmup_steps, 1)
        else:
            progress = (step - warmup_steps) / max(total_steps - warmup_steps, 1)
            return max(0.5 * (1 + np.cos(np.pi * progress)), 1e-6 / config['learning_rate'])
    
    scheduler = optim.lr_scheduler.LambdaLR(optimizer, lr_lambda)
    scaler = GradScaler('cuda')
    
    # SWA setup
    swa_model = None
    swa_scheduler = None
    if config['use_swa']:
        swa_model = AveragedModel(model)
        swa_scheduler = SWALR(optimizer, swa_lr=config['swa_lr'])
    
    early_stopping = EarlyStopping(patience=config['patience'], min_delta=0.003, mode='max')
    
    best_balanced_acc = 0.0
    history = {
        'train_loss': [], 'train_acc': [],
        'val_loss': [], 'val_acc': [], 'balanced_acc': [],
        'class_0_recall': [], 'class_1_recall': [], 'class_2_recall': []
    }
    
    print(f"\n🚀 Training Fold {fold+1}/{config['num_folds']} (v5)")
    print(f"   Train: {len(train_loader.dataset)}, Val: {len(val_loader.dataset)}")
    print(f"   Focal Gamma: {config['focal_gamma']}, Smoothing: {config['label_smoothing']}")
    print(f"   Grad Clip: {config['clip_grad_norm']}, Mixup: {config['use_mixup']}")
    print(f"   SWA: starts epoch {config['swa_start_epoch']}")
    print(f"   ⚠️  Model saved based on BALANCED ACCURACY")
    
    for epoch in range(config['epochs']):
        model.train()
        train_loss = 0
        train_correct = 0
        train_total = 0
        is_swa_phase = config['use_swa'] and epoch >= config['swa_start_epoch']
        
        loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/{config['epochs']}" + (" [SWA]" if is_swa_phase else ""))
        
        for images, labels, level_idx in loop:
            images = images.to(config['device'])
            labels = labels.to(config['device'])
            level_idx = level_idx.to(config['device'])
            
            # Mixup (Tier 3)
            use_mixup = config['use_mixup'] and not is_swa_phase and random.random() < 0.5
            if use_mixup:
                images, labels_a, labels_b, lam = mixup_data(images, labels, config['mixup_alpha'])
            
            optimizer.zero_grad()
            
            with autocast('cuda'):
                logits, _ = model(images, level_idx)
                if use_mixup:
                    loss = mixup_criterion(criterion, logits, labels_a, labels_b, lam)
                else:
                    loss = criterion(logits, labels)
                
            scaler.scale(loss).backward()
            
            # Gradient clipping (Tier 1)
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(model.parameters(), config['clip_grad_norm'])
            
            scaler.step(optimizer)
            scaler.update()
            
            if is_swa_phase:
                swa_scheduler.step()
            else:
                scheduler.step()
            
            train_loss += loss.item()
            _, predicted = torch.max(logits, 1)
            if use_mixup:
                train_correct += (lam * (predicted == labels_a).float() + 
                                  (1 - lam) * (predicted == labels_b).float()).sum().item()
            else:
                train_correct += (predicted == labels).sum().item()
            train_total += labels.size(0)
            
            loop.set_postfix(
                loss=f"{train_loss/(loop.n+1):.4f}", 
                acc=f"{100*train_correct/train_total:.1f}%",
                lr=f"{optimizer.param_groups[0]['lr']:.2e}"
            )
        
        train_epoch_loss = train_loss / len(train_loader)
        train_acc = train_correct / train_total
        
        # Update SWA model
        if is_swa_phase and swa_model is not None:
            swa_model.update_parameters(model)
        
        # Validation
        eval_model = swa_model if (is_swa_phase and swa_model is not None) else model
        eval_model.eval()
        val_loss = 0
        val_correct = 0
        val_total = 0
        all_preds = []
        all_labels = []
        
        with torch.no_grad():
            for images, labels, level_idx in val_loader:
                images = images.to(config['device'])
                labels = labels.to(config['device'])
                level_idx = level_idx.to(config['device'])
                
                with autocast('cuda'):
                    if is_swa_phase and swa_model is not None:
                        logits = swa_model(images, level_idx)
                        if isinstance(logits, tuple):
                            logits = logits[0]
                    else:
                        logits, _ = model(images, level_idx)
                    loss = criterion(logits, labels)
                
                val_loss += loss.item()
                _, predicted = torch.max(logits, 1)
                val_correct += (predicted == labels).sum().item()
                val_total += labels.size(0)
                
                all_preds.extend(predicted.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())
        
        val_epoch_loss = val_loss / len(val_loader)
        val_acc = val_correct / val_total
        
        # Per-class metrics
        all_preds = np.array(all_preds)
        all_labels = np.array(all_labels)
        per_class = compute_per_class_metrics(all_preds, all_labels)
        
        balanced_acc = (per_class['class_0_recall'] + 
                       per_class['class_1_recall'] + 
                       per_class['class_2_recall']) / 3
        
        history['train_loss'].append(train_epoch_loss)
        history['train_acc'].append(train_acc)
        history['val_loss'].append(val_epoch_loss)
        history['val_acc'].append(val_acc)
        history['balanced_acc'].append(balanced_acc)
        history['class_0_recall'].append(per_class['class_0_recall'])
        history['class_1_recall'].append(per_class['class_1_recall'])
        history['class_2_recall'].append(per_class['class_2_recall'])
        
        print(f"📊 Train Loss: {train_epoch_loss:.4f} | Train Acc: {100*train_acc:.1f}% | "
              f"Val Loss: {val_epoch_loss:.4f} | Val Acc: {100*val_acc:.1f}%")
        print(f"   Per-class Recall: Normal={100*per_class['class_0_recall']:.1f}%, "
              f"Moderate={100*per_class['class_1_recall']:.1f}%, "
              f"Severe={100*per_class['class_2_recall']:.1f}%")
        print(f"   🎯 Balanced Accuracy: {100*balanced_acc:.1f}%"
              f"{' [SWA]' if is_swa_phase else ''}")
        
        # Save best model
        min_minority_recall = min(per_class['class_1_recall'], per_class['class_2_recall'])
        
        if balanced_acc > best_balanced_acc and min_minority_recall >= config.get('min_minority_recall', 0.1):
            best_balanced_acc = balanced_acc
            save_dict = swa_model.module.state_dict() if (is_swa_phase and swa_model is not None) else model.state_dict()
            torch.save(save_dict, f"best_model_v5_fold{fold}.pth")
            print(f"✅ Best Model Saved! (BA: {100*balanced_acc:.1f}%, "
                  f"Min Minority: {100*min_minority_recall:.1f}%)")
        
        if early_stopping(balanced_acc):
            print(f"⏹️ Early stopping at epoch {epoch+1}")
            break
    
    # Load best model
    model.load_state_dict(torch.load(f"best_model_v5_fold{fold}.pth"))
    
    return model, history, best_balanced_acc


## 8. Training with Weighted Sampling

In [ ]:
kfold = StratifiedGroupKFold(n_splits=CONFIG['num_folds'], shuffle=True, random_state=CONFIG['seed'])
fold_results = []


In [ ]:
for fold, (train_idx, val_idx) in enumerate(kfold.split(df_final, df_final['label'], df_final['study_id'])):
    if fold not in CONFIG['train_folds']:
        continue
    
    print(f"\n{'='*60}")
    print(f"FOLD {fold + 1}/{CONFIG['num_folds']} (v5)")
    print(f"{'='*60}")
    
    train_df = df_final.iloc[train_idx].reset_index(drop=True)
    val_df = df_final.iloc[val_idx].reset_index(drop=True)
    
    # Class distribution
    print(f"\n📊 Class Distribution:")
    for i in range(3):
        count = (train_df['label'] == i).sum()
        print(f"   Class {i}: {count} samples ({100*count/len(train_df):.1f}%)")
    
    # Create weighted sampler (replaces oversampling!)
    sampler = create_weighted_sampler(train_df)
    
    # Datasets
    train_dataset = RSNASequenceDatasetV5(
        train_df, 
        seq_length=CONFIG['seq_length'], 
        img_size=CONFIG['img_size'], 
        transform=train_aug,
        is_training=True
    )
    
    val_dataset = RSNASequenceDatasetV5(
        val_df, 
        seq_length=CONFIG['seq_length'], 
        img_size=CONFIG['img_size'], 
        transform=val_aug,
        is_training=False
    )
    
    # Note: sampler replaces shuffle=True
    train_loader = DataLoader(
        train_dataset, 
        batch_size=CONFIG['batch_size'], 
        sampler=sampler,  # Balanced sampling!
        num_workers=2,
        pin_memory=True,
        drop_last=True
    )
    val_loader = DataLoader(
        val_dataset, 
        batch_size=CONFIG['batch_size'], 
        shuffle=False, 
        num_workers=2,
        pin_memory=True
    )
    
    # Create model
    model = SpineModelV5(
        num_classes=3,
        hidden_dim=CONFIG['hidden_dim'],
        dropout=CONFIG['dropout'],
        stochastic_depth=CONFIG['stochastic_depth_rate']
    ).to(CONFIG['device'])
    
    param_count = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"\n🏗️  Model: SpineModelV5 ({param_count:,} trainable params)")
    
    # Train
    model, history, best_balanced_acc = train_one_fold_v5(
        model, train_loader, val_loader, fold, CONFIG
    )
    
    fold_results.append({
        'fold': fold,
        'best_balanced_acc': best_balanced_acc,
        'history': history
    })
    
    print(f"\n✅ Fold {fold+1} Complete | Best Balanced Acc: {100*best_balanced_acc:.1f}%")


In [ ]:
print("\n" + "="*60)
print("TRAINING SUMMARY")
print("="*60)
for r in fold_results:
    print(f"Fold {r['fold']+1}: Best Balanced Acc = {100*r['best_balanced_acc']:.1f}%")


## 9. Evaluation with Test-Time Augmentation (TTA)

TTA averages predictions over multiple augmentations of the same input.
Even a simple horizontal flip typically adds +1-2% balanced accuracy.


In [ ]:
def predict_with_tta(model, dataset, config, tta_augs):
    """Run inference with test-time augmentation."""
    model.eval()
    all_probs = []
    all_labels = []
    
    for aug_idx, aug in enumerate(tta_augs):
        # Create dataset with this augmentation
        tta_dataset = RSNASequenceDatasetV5(
            dataset.df,
            seq_length=config['seq_length'],
            img_size=config['img_size'],
            transform=aug,
            is_training=False
        )
        loader = DataLoader(tta_dataset, batch_size=config['batch_size'], 
                          shuffle=False, num_workers=2, pin_memory=True)
        
        aug_probs = []
        aug_labels = []
        
        with torch.no_grad():
            for images, labels, level_idx in loader:
                images = images.to(config['device'])
                level_idx = level_idx.to(config['device'])
                
                with autocast('cuda'):
                    logits, _ = model(images, level_idx)
                    probs = F.softmax(logits, dim=1)
                
                aug_probs.append(probs.cpu().numpy())
                if aug_idx == 0:
                    aug_labels.extend(labels.numpy())
        
        all_probs.append(np.concatenate(aug_probs, axis=0))
        if aug_idx == 0:
            all_labels = np.array(aug_labels)
    
    # Average probabilities across TTA augmentations
    avg_probs = np.mean(all_probs, axis=0)
    avg_preds = np.argmax(avg_probs, axis=1)
    
    return avg_preds, all_labels, avg_probs

print("✅ TTA inference function ready")


In [ ]:
# Run TTA evaluation
model.eval()
tta_preds, tta_labels, tta_probs = predict_with_tta(model, val_dataset, CONFIG, tta_augs)

# Results without TTA
no_tta_preds, _, _ = predict_with_tta(model, val_dataset, CONFIG, [val_aug])

# Compare
per_class_no_tta = compute_per_class_metrics(no_tta_preds, tta_labels)
ba_no_tta = np.mean([per_class_no_tta[f'class_{c}_recall'] for c in range(3)])

per_class_tta = compute_per_class_metrics(tta_preds, tta_labels)
ba_tta = np.mean([per_class_tta[f'class_{c}_recall'] for c in range(3)])

print(f"\n{'='*60}")
print(f"RESULTS COMPARISON")
print(f"{'='*60}")
print(f"\nWithout TTA:")
print(f"   Balanced Accuracy: {100*ba_no_tta:.1f}%")
print(f"   Normal:   {100*per_class_no_tta['class_0_recall']:.1f}%")
print(f"   Moderate: {100*per_class_no_tta['class_1_recall']:.1f}%")
print(f"   Severe:   {100*per_class_no_tta['class_2_recall']:.1f}%")

print(f"\nWith TTA ({len(tta_augs)} augmentations):")
print(f"   Balanced Accuracy: {100*ba_tta:.1f}%")
print(f"   Normal:   {100*per_class_tta['class_0_recall']:.1f}%")
print(f"   Moderate: {100*per_class_tta['class_1_recall']:.1f}%")
print(f"   Severe:   {100*per_class_tta['class_2_recall']:.1f}%")
print(f"\n   TTA improvement: {100*(ba_tta - ba_no_tta):+.1f}%")


In [ ]:
print("\n" + "="*50)
print("CLASSIFICATION REPORT (with TTA)")
print("="*50)
print(classification_report(tta_labels, tta_preds, 
                           target_names=['Normal/Mild', 'Moderate', 'Severe']))

# Confusion matrix
cm = confusion_matrix(tta_labels, tta_preds)
cm_norm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
plt.figure(figsize=(8, 6))
sns.heatmap(cm_norm, annot=True, fmt='.2%', cmap='Blues',
            xticklabels=['Normal/Mild', 'Moderate', 'Severe'],
            yticklabels=['Normal/Mild', 'Moderate', 'Severe'])
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.title(f'Confusion Matrix (BA: {100*ba_tta:.1f}%)')
plt.tight_layout()
plt.show()


In [ ]:
# Plot training history
if fold_results:
    history = fold_results[0]['history']
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    epochs = range(1, len(history['train_loss']) + 1)
    
    # Loss
    axes[0].plot(epochs, history['train_loss'], 'b-', label='Train')
    axes[0].plot(epochs, history['val_loss'], 'r-', label='Val')
    axes[0].set_xlabel('Epoch')
    axes[0].set_ylabel('Loss')
    axes[0].set_title('Loss')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # Per-class recall
    axes[1].plot(epochs, history['class_0_recall'], 'g-o', label='Normal', markersize=3)
    axes[1].plot(epochs, history['class_1_recall'], 'orange', marker='s', label='Moderate', markersize=3)
    axes[1].plot(epochs, history['class_2_recall'], 'r-^', label='Severe', markersize=3)
    axes[1].set_xlabel('Epoch')
    axes[1].set_ylabel('Recall')
    axes[1].set_title('Per-Class Recall')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    # Balanced accuracy
    axes[2].plot(epochs, history['balanced_acc'], 'purple', marker='d', linewidth=2, markersize=3)
    axes[2].set_xlabel('Epoch')
    axes[2].set_ylabel('Balanced Accuracy')
    axes[2].set_title(f'Balanced Accuracy (Best: {100*max(history["balanced_acc"]):.1f}%)')
    axes[2].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()


In [ ]:
print("\n" + "="*60)
print("TRAINING COMPLETE — Version 5 (Redesigned Pipeline)")
print("="*60)
print(f"\nKey improvements in v5:")
print(f"  ✓ WeightedRandomSampler (no oversampling duplicates)")
print(f"  ✓ DICOM windowing (clinical contrast preserved)")
print(f"  ✓ Attention pooling (learns important frames)")
print(f"  ✓ FiLM conditioning (level modulates features)")
print(f"  ✓ Gradient clipping ({CONFIG['clip_grad_norm']})")
print(f"  ✓ Focal loss without class weights (sampler handles balance)")
print(f"  ✓ Mixup augmentation")
print(f"  ✓ SWA in final epochs")
print(f"  ✓ TTA at inference")
print(f"\n🎯 Target: 80-85%+ Balanced Accuracy")
